In [46]:
import pandas as pd
import json
import os
from etl import *
import hashlib

In [49]:
def get_updated_mbf_file_paths(file_paths):
    file_hash = pd.DataFrame([[file_path,get_file_hash(file_path)] for file_path in file_paths]).rename({0:'file_path',1:'hash'},axis=1)
    file_hash['is_duplicated'] = file_hash['hash'].duplicated()
    dupelicated_df = file_hash[file_hash['is_duplicated']==True].reset_index(drop=True)
    if os.path.isfile(config['excel_file_dir']):
        excel_df = pd.read_excel(config['excel_file_dir'])
        excel_hash = excel_df['hash'].drop_duplicates().reset_index().rename({'index':'is_exists'},axis=1)
        hash_df = file_hash.merge( excel_hash, how='left', on='hash',suffixes=('_left', '_right'))
        hash_df['is_exists'] = hash_df['is_exists'].apply(lambda x: False if pd.isna(x) else True)
        updated_df = hash_df[(hash_df['is_duplicated']==False) & (hash_df['is_exists']==False)].reset_index(drop=True)
        not_updated_df = hash_df[(hash_df['is_duplicated']==False) & (hash_df['is_exists']==True)].reset_index(drop=True)
        not_updated_excel_df = excel_df[excel_df['hash'].isin(not_updated_df['hash'])]
        return dupelicated_df['file_path'].to_list(), updated_df['file_path'].to_list(), not_updated_excel_df
    else:
        return dupelicated_df['file_path'].to_list(), file_hash[file_hash['is_duplicated']==False]['file_path'].to_list(), pd.DataFrame()


In [89]:
if os.path.isfile("./config.json"):
    with open("./config.json") as config_file:
        config = json.load(config_file)
file_paths = get_mbf_file_paths("./mbf_files")
dupelicated_file_paths, updated_file_paths, not_updated_excel_df = get_updated_mbf_file_paths(file_paths)

In [54]:
updated_file_paths

['./mbf_files\\اردیبهشت\\140202-100-کسری شجاعی.mbf']

In [57]:
general_dataframe = get_dataframes(updated_file_paths)
general_dataframe = pd.concat([not_updated_excel_df, general_dataframe]).reset_index(drop=True)

File  1 processed.


In [80]:
general_dataframe['order_id'] = pd.factorize(general_dataframe['hash'])[0]+1

In [90]:
if 1==1:
    file_hash = pd.DataFrame([[file_path,get_file_hash(file_path)] for file_path in file_paths]).rename({0:'file_path',1:'hash'},axis=1)
    file_hash['is_duplicated'] = file_hash['hash'].duplicated()
    dupelicated_df = file_hash[file_hash['is_duplicated']==True].reset_index(drop=True)
    if os.path.isfile(config['excel_file_dir']):
        excel_df = pd.read_excel(config['excel_file_dir'])
        excel_hash = excel_df['hash'].drop_duplicates().reset_index().rename({'index':'is_exists'},axis=1)
        hash_df = file_hash.merge( excel_hash, how='left', on='hash',suffixes=('_left', '_right'))
        hash_df['is_exists'] = hash_df['is_exists'].apply(lambda x: False if pd.isna(x) else True)
        updated_df = hash_df[(hash_df['is_duplicated']==False) & (hash_df['is_exists']==False)].reset_index(drop=True)
        not_updated_df = hash_df[(hash_df['is_duplicated']==False) & (hash_df['is_exists']==True)].reset_index(drop=True)
        not_updated_excel_df = excel_df[excel_df['hash'].isin(not_updated_df['hash'])]
        deleted_df = excel_df.merge(file_hash, how='left', on='hash', suffixes=(None,'_y'))
        deleted = deleted_df[pd.isna(deleted_df['file_path_y'])]['hash'].to_list()

In [ ]:
excel_df